In [1]:
import numpy as np
import pandas as pd

In [15]:
#input data
resume_cleaned = 'C:/Users/ashua/Desktop/Inelligent Job Recomendation Engine/data/Resumes/resume_cleaned.csv'
job_skills_cleaned = 'C:/Users/ashua/Desktop/Inelligent Job Recomendation Engine/data/Jobs/job_skills_cleaned.csv'

In [4]:
#output files
resume_skll_features_path = 'resume_skill_features.csv'
job_skill_features_path = 'job_skill_features.csv'

In [5]:
# Definitive 35 Skill Codes
SKILL_CODES = [
    'ACCT', 'ADM', 'ADVR', 'ANLS', 'ART', 'BD', 'CNST', 'DSGN', 'EDCN', 'ENG',
    'FASH', 'FIN', 'GENB', 'HCPR', 'HR', 'IT', 'LGL', 'MGMT', 'MNFC', 'MRKT',
    'OTHR', 'PR', 'PRJM', 'PROD', 'PRSR', 'QA', 'REAL', 'RSCH', 'SALE', 'SCI',
    'SPRT', 'SUPL', 'TECH', 'TRNS', 'WRT'
]

In [ ]:
# Manual mapping from broad resume category (long-form) to the single primary short skill code
CATEGORY_TO_CODE = {
    'ACCOUNTANT': 'ACCT', 'ADVOCATE': 'LGL', 'AGRICULTURE': 'SCI', 'APPAREL': 'FASH',
    'ARTS': 'ART', 'AUTOMOBILE': 'ENG', 'BANKING': 'FIN', 'BUSINESS-DEVELOPMENT': 'BD',
    'CHEF': 'OTHR', 'CONSTRUCTION': 'CNST', 'CONSULTANT': 'CNST', 'DESIGNER': 'DSGN',
    'DIGITAL-MEDIA': 'ADVR', 'ENGINEERING': 'ENG', 'FINANCE': 'FIN', 'FITNESS': 'HCPR',
    'HR': 'HR', 'HEALTHCARE': 'HCPR', 'INFORMATION-TECHNOLOGY': 'IT', 'PUBLIC-RELATIONS': 'PR',
    'SALES': 'SALE', 'TEACHER': 'EDCN', 'OTHERS': 'OTHR', 'ADV_ADV': 'ADVR',
    'MANAGEMENT': 'MGMT', 'SCIENTIST': 'SCI', 'MARKETING': 'MRKT', 'BUSINESS': 'GENB',
    'LEGAL': 'LGL', 'RESEARCH': 'RSCH', 'WRITER': 'WRT', 'ADMINISTRATION': 'ADM',
    'QUALITY-ASSURANCE': 'QA', 'REAL-ESTATE': 'REAL', 'SUPPLY-CHAIN': 'SUPL',
    'PROJECT-MANAGEMENT': 'PRJM', 'TECHNICAL': 'TECH', 'MANUFACTURING': 'MNFC',
    'TRANSPORTATION': 'TRNS', 'PRODUCTION': 'PROD', 'PRESS-RELATIONS': 'PRSR',
    'ANALYSIS': 'ANLS', 'SPORTS': 'SPRT',
}

print("✅ Setup complete. Constants defined.")

✅ Setup complete. Constants defined.


In [13]:
#Generate Resume Skill Features
print(f"Loading resume data from {resume_cleaned}...")
try:
    resume_df = pd.read_csv(resume_cleaned)
except FileNotFoundError:
    print(f"Error: The file {resume_cleaned} was not found.")
    raise

# Initialize the resume feature matrix with 0s
resume_features = pd.DataFrame(0, index=resume_df['ID'], columns=SKILL_CODES)

#Apply the category to code mapping
for index, row in resume_df.iterrows():
    resume_id = row['ID']
    category = str(row['Category']).replace('-', ' ').replace('_', ' ').upper()

    found_code = None
    for k, v in CATEGORY_TO_CODE.items():
        if k.replace('-', ' ').replace('_', ' ') in category:
            found_code = v
            break
    
    if found_code and found_code in SKILL_CODES:
        resume_features.at[resume_id, found_code] = 1

# Save the resume skill features to CSV
resume_features_output = resume_features.reset_index().rename(columns={'index': 'ID'})
resume_features_output.to_csv(resume_skll_features_path, index=False)

print(f"Resume skill features saved to {resume_skll_features_path}.")
print(resume_features_output.shape)
resume_features_output.head()


Loading resume data from C:/Users/ashua/Desktop/Inelligent Job Recomendation Engine/data/Resumes/resume_cleaned.csv...
Resume skill features saved to resume_skill_features.csv.
(2484, 36)


,ID,ACCT,ADM,ADVR,ANLS,ART,BD,CNST,DSGN,EDCN,...,QA,REAL,RSCH,SALE,SCI,SPRT,SUPL,TECH,TRNS,WRT
0,16852973,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22323967,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,33176873,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,27018550,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17812897,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Generate Job Skill Features
print(f"Loading job data from {job_skills_cleaned}...")
try:
    job_df = pd.read_csv(job_skills_cleaned)
except FileNotFoundError:
    print(f"Error: The file {job_skills_cleaned} was not found.")
    raise

#Pivot to wide format
job_features_wide = job_df.pivot_table(index='job_id', columns='skill_abr', aggfunc='size', fill_value=0)

# Conver to binary presence/absence
job_features_wide = (job_features_wide > 0).astype(int)

#Reindex to ensure all skill codes are present
job_features_output = job_features_wide.reindex(columns=SKILL_CODES, fill_value=0).reset_index()

# Save the job skill features to CSV
job_features_output.to_csv(job_skill_features_path, index=False)

print(f"Job skill features saved to {job_skill_features_path}.")    
print(job_features_output.shape)
job_features_output.head()


Loading job data from C:/Users/ashua/Desktop/Inelligent Job Recomendation Engine/data/Jobs/job_skills_cleaned.csv...
Job skill features saved to job_skill_features.csv.
(126807, 36)


skill_abr,job_id,ACCT,ADM,ADVR,ANLS,ART,BD,CNST,DSGN,EDCN,...,QA,REAL,RSCH,SALE,SCI,SPRT,SUPL,TECH,TRNS,WRT
0,921716,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1218575,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1829192,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2264355,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,10998357,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
